In [12]:
# 通过token编码，最大铸造量的代币锁定。
class LPool:
    # 先简单点，假设权重都一样
    def __init__(self, rD, rF): #, weight=1):
        self.reserve_D = rD # 允许支付D的余额
        self.reserve_F = rF # 允许支付F的余额
        self.supply_D = rD  # 该池收集的D的总量
        self.supply_F = rF  # 该池铸造的F的总量（包括发行成本）
        # 限定铸造上限
        # 通过权重限定
        self.limit_F = self.reserve_F+self.upply_F 
        # self.weight = 1
    
    def getNetPrice(self):
        return self.sD/self.sF


In [3]:
class AMM:
    def __init__(self):
    self.pools = [] # 所有的流动性池
    self.totalSupply = 1 # 总铸造量
    # # biggest_supply_div_weight => bsdw
    # self.bsdw = 0 # 最大铸造量的池子的supply/权重
    # self.bsdw_id = 0 # 加权最大铸造量的池子的id
    self.biggest_supply = 0 # 最大铸造量的池子的supply
    self.biggest_supply_id = 0 # 最大铸造量的池子的id
    # 剔除最大潜在损失折扣
    self.dis = 1 - self.biggest_supply/self.totalSupply
    
    def addPool(self, reserve_D, reserve_F):
        self.pools.append(LPool(rD, rF))
        return len(self.pools)-1

    # def fillLiquidity(self, id):
    #     pool = self.pools[id]
    #     net = self.getNetPrice(id)
    #     rD = self.limitrF*net
    #     if rD < self.sD:
    #         self.rF = self.limitrF
    #         self.rD = rD
    #         # print('fill liquidity success')
    #     else:
    #         self.rF = self.sD/net
    #         self.rD = self.sD
    #         # print('fill liquidity failed')
        
    def buyF(self, id, n):
        pool = self.pools[id]
        max_sf = max([pool.sF for pool in self.pools])
        pool.rF *= max_sf
        if pool.rF > n > 0:
            # exchange
            K = pool.rD*pool.rF
            self.rF -= n
            rD = K/self.rF
            incomeD = rD - self.rD
            # supply change
            self.sD += incomeD
            self.sF += (n*0.9985) # 0.15% fee
            # net asset change
            
            # [1]repair by reduce rD
            # net = self.sD/self.sF
            # self.rD = self.rF * net
            # [2]do not repair
            self.rD = rD
            return True
        else:
            return False
    
    def sellF(self, n):
        # assert n > 0
        # assert self.sF > n # 可能市面上都没有足够的F了 
        # 如果该池铸出的流动性全部收回了，此处应当如何处理？
        if self.sF > n >0:
            # exchange
            K = self.rD*self.rF
            rF = self.rF + n
            rD = K/rF
            outD = self.rD - rD
            # supply change
            self.sD -= (outD*0.9985) # 0.15% fee
            self.sF -= n
            # net asset change TODO--
            net = self.sD/self.sF
            # [1] repair by raise rD first？
            # if rF * net < self.sD:
            #     self.rF = rF
            #     self.rD = rF * net
            # else:
            #     # repair by reduce rF
            #     self.rD = rD
            #     self.rF = rD / net
            # [2] only repair by reduce rF
            self.rD = rD
            self.rF = rD / net
            return True
        else:
            return False
    
    def sell_price(self, n):
        if self.sF > n > 0:
            K = self.rD*self.rF
            rF = self.rF + n
            rD = K/rF
            outD = self.rD - rD
            return outD*0.9985
        else:
            return 0

In [ ]:
class Pool:
    def __init__(self, rD, rF):
        self.rD = rD
        self.rF = rF
        self.sD = rD
        self.sF = rF
        self.limitrF = rF
        
    def getNetPrice(self):
        return self.sD/self.sF
        
    def getNetPrice(self, id):
        pool = self.pools[id]
        max_sf = max([pool.sF for pool in self.pools])
        pool.rF *= (max_sf/pool.sF)
        return self.pools[id].getNetPrice()
    
    # def fillLiquidity(self, id):
    #     pool = self.pools[id]
    #     net = self.getNetPrice(id)
    #     rD = self.limitrF*net
    #     if rD < self.sD:
    #         self.rF = self.limitrF
    #         self.rD = rD
    #         # print('fill liquidity success')
    #     else:
    #         self.rF = self.sD/net
    #         self.rD = self.sD
    #         # print('fill liquidity failed')
        
    def buyF(self, id, n):
        pool = self.pools[id]
        max_sf = max([pool.sF for pool in self.pools])
        pool.rF *= max_sf
        if pool.rF > n > 0:
            # exchange
            K = pool.rD*pool.rF
            self.rF -= n
            rD = K/self.rF
            incomeD = rD - self.rD
            # supply change
            self.sD += incomeD
            self.sF += (n*0.9985) # 0.15% fee
            # net asset change
            
            # [1]repair by reduce rD
            # net = self.sD/self.sF
            # self.rD = self.rF * net
            # [2]do not repair
            self.rD = rD
            return True
        else:
            return False
    
    def sellF(self, n):
        # assert n > 0
        # assert self.sF > n # 可能市面上都没有足够的F了 
        # 如果该池铸出的流动性全部收回了，此处应当如何处理？
        if self.sF > n >0:
            # exchange
            K = self.rD*self.rF
            rF = self.rF + n
            rD = K/rF
            outD = self.rD - rD
            # supply change
            self.sD -= (outD*0.9985) # 0.15% fee
            self.sF -= n
            # net asset change TODO--
            net = self.sD/self.sF
            # [1] repair by raise rD first？
            # if rF * net < self.sD:
            #     self.rF = rF
            #     self.rD = rF * net
            # else:
            #     # repair by reduce rF
            #     self.rD = rD
            #     self.rF = rD / net
            # [2] only repair by reduce rF
            self.rD = rD
            self.rF = rD / net
            return True
        else:
            return False